In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

path_data ='//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/'
path_las = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/well logs/'
path_wvlt = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/wavelets/'
fig_path = 'M:/LOCRETA_WORKPACKAGE_D/Valdemar_sprint_well_data_Mads_L/MLO documents/Figures/'
path_hrs = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/HRS project/Locreta_proj_Mads_localtest.prj/seismic.dir/'

plt.style.use('default')
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['legend.fontsize'] = 'small'
matplotlib.rcParams['axes.labelsize'] = 'medium'
plt.rc('xtick',labelsize=14)
plt.rc('ytick',labelsize=14)

from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import feature_selection
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from scikeras.wrappers import KerasRegressor

# Splitting data

In [ ]:
path_project = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/well logs/locreta_ml/'
df= pd.read_csv(path_project+'wells_merged_new.csv')
df

In [ ]:
features = ['GRN',
 'Vp',
 'CNC',
 'ILM_log10']

target = ['Vs']

train_wells = ['JUDE-1','VALDEMAR-2H', 'BOJE-2C','SIAH_NE-01X']

X_train = df.loc[df['UWI'].isin(train_wells), features].values
y_train = df.loc[df['UWI'].isin(train_wells), target].values

# And we might need this later...
wells_train = df.loc[df['UWI'].isin(train_wells), 'UWI'].values

test_wells = ['BO-2X', 'BO-3X']

X_test = df.loc[df['UWI'].isin(test_wells), features].values
y_test = df.loc[df['UWI'].isin(test_wells), target].values
len(features)

test_logs = pd.DataFrame(y_test, columns = ['true'])

#plt.figure(figsize=(10, 5))
#ax = df.groupby('UWI').NEUT.count().plot(kind='bar', color=['C0', 'C1', 'C2', 'C3', 'C4', 'C5'])
#plt.ylim(0,1300)
#plt.xticks(rotation = 0, fontsize = 13)
#plt.xlabel('Well name')
#plt.ylabel('Counts')
#for p in ax.patches:
#    ax.annotate(str(p.get_height()),(p.get_x()+p.get_width()/2., 
#                                     p.get_height()),    ha='center',   va='center',   xytext=(0, 10),  textcoords='offset points') 
#plt.savefig(fig_path+'wells_counts.png',format='png', bbox_inches='tight') 

In [ ]:
#plt.scatter(x=df['DTS'],y = df['DENS'], c = df['DEPTH'], cmap  = 'viridis')
#plt.colorbar()

# Setting up pipelines

In [ ]:
regr1 = LinearRegression()
regr2 = RandomForestRegressor(random_state=42, n_jobs=5)
regr3 = SVR()
regr4 = MLPRegressor(max_iter=1000, random_state=42, early_stopping=True, validation_fraction=0.2)



# Building the pipelines
pipe_LR = Pipeline([('std', StandardScaler()),
                  ('LR', regr1)])

pipe_RFR = Pipeline([('RFR', regr2)])

pipe_SVR = Pipeline([('std', StandardScaler()),
                 ('SVR', regr3)])

pipe_MLP = Pipeline([('std', StandardScaler())
                 , ('MLP', regr4)])

# Setting up the parameter grids
param_grid_LR = [{}]

param_grid_RFR = [{ 
 'RFR__max_depth': [5, 8 ,10 ,15 ],
 'RFR__min_samples_leaf': [ 8, 10, 12],
 'RFR__min_samples_split': [2,3,4],
 'RFR__n_estimators': [600 ,800, 1000] }]


param_grid_SVR = [{'SVR__kernel' : ['linear','rbf'],
                    'SVR__C': np.power(10., np.arange(-4, 4)),
                  'SVR__gamma': [1, 0.1, 0.01, 0.001, 0.0001]}]

param_grid_MLP = [{"MLP__hidden_layer_sizes": [(8,8), (14,14), (20, 20)],
                 "MLP__activation": ["relu"],
                 'MLP__learning_rate': ['constant','adaptive'],
                 'MLP__solver' : ['lbfgs','adam'],
                 "MLP__alpha": np.power(10., np.arange(-6, 0)),
                 "MLP__max_iter": [4000],
                  "MLP__n_iter_no_change":[10]}]

# Nested CV

In [ ]:
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

X_tr = df.loc[df['UWI'].isin(train_wells), features]
y_tr = df.loc[df['UWI'].isin(train_wells), target]
X_ts = df.loc[df['UWI'].isin(test_wells), features]
y_ts = df.loc[df['UWI'].isin(test_wells), target]

logo = LeaveOneGroupOut()
k=0

nested_score_LR = []
nested_score_RFR = []
nested_score_SVR = []
nested_score_MLP = []

for train_index, test_index in logo.split(X_tr, y_tr, wells_train):
    print('Iter #:', k)
    X_trn, X_tst = X_tr.iloc[train_index], X_tr.iloc[test_index]
    y_trn, y_tst = y_tr.iloc[train_index], y_tr.iloc[test_index]
    
    print('test well :', df.loc[X_tst.index[-1]]['UWI'])


    wells_train_inner = train_wells.copy()
    wells_train_inner.remove(df.loc[X_tst.index[-1]]['UWI'])
    
    print('training wells:', wells_train_inner)
    
    X_tr_ = df.loc[df['UWI'].isin(wells_train_inner), features].values
    y_tr_ = np.squeeze(df.loc[df['UWI'].isin(wells_train_inner), target].values)


    wells_train_inner = df.loc[df['UWI'].isin(wells_train_inner), 'UWI'].values
    
    #LR
    gcv_LR = GridSearchCV(pipe_LR, param_grid_LR, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut(), n_jobs=-1)
    results_LR = gcv_LR.fit(X_tr_, y_tr_, groups=wells_train_inner)
    model_LR = results_LR.best_estimator_.fit(X_tr_, y_tr_)
    model_score_LR = model_LR.predict(X_tst)
    nested_score_LR.append(mean_squared_error(y_tst,model_score_LR, squared=False))
    
    #RFR
    gcv_RFR = GridSearchCV(pipe_RFR, param_grid_RFR, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut(), n_jobs=-1)
    results_RFR = gcv_RFR.fit(X_tr_, y_tr_, groups=wells_train_inner)
    print(results_RFR.best_estimator_)
    model_RFR = results_RFR.best_estimator_.fit(X_tr_, y_tr_)
    model_score_RFR = model_RFR.predict(X_tst)
    nested_score_RFR.append(mean_squared_error(y_tst,model_score_RFR, squared=False))
    
    
    #SVR
    gcv_SVR = GridSearchCV(pipe_SVR, param_grid_SVR, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut(), n_jobs=-1)
    results_SVR = gcv_SVR.fit(X_tr_, y_tr_, groups=wells_train_inner)
    print(results_SVR.best_estimator_)
    model_SVR = results_SVR.best_estimator_.fit(X_tr_, y_tr_)
    model_score_SVR = model_SVR.predict(X_tst)
    nested_score_SVR.append(mean_squared_error(y_tst,model_score_SVR, squared=False))
    
    #MLP
    gcv_MLP = GridSearchCV(pipe_MLP, param_grid_MLP, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut(), n_jobs=-1)
    results_MLP = gcv_MLP.fit(X_tr_, y_tr_, groups=wells_train_inner)
    print(results_MLP.best_estimator_)
    model_MLP = results_MLP.best_estimator_.fit(X_tr_, y_tr_)
    model_score_MLP = model_MLP.predict(X_tst)
    nested_score_MLP.append(mean_squared_error(y_tst,model_score_MLP, squared=False))
    
    k+=1

In [ ]:
nested_cv = pd.DataFrame(data = np.array([nested_score_LR, nested_score_RFR, nested_score_SVR, nested_score_MLP]).T, columns = ['LR','RFR','SVR', 'MLP'])
nested_cv.to_csv(path_project+'nested_cv.csv',index=False)

In [ ]:
nested_cv= pd.read_csv(path_project+'nested_cv.csv')

In [ ]:
nested_cv

In [ ]:
plt.figure(figsize = (8,6))
plt.plot(np.arange(4), nested_cv['LR']  , label = 'LR   (RMSE mean score: {})'.format(np.round(np.mean(nested_cv['LR'] ),2)))
plt.plot(np.arange(4), nested_cv['RFR'] , label = 'RFR (RMSE mean score: {})'.format( np.round(np.mean(nested_cv['RFR']),2)))
plt.plot(np.arange(4), nested_cv['SVR'] , label = 'SVR (RMSE mean score:{})'.format(  np.round(np.mean(nested_cv['SVR']),2)))
plt.plot(np.arange(4), nested_cv['MLP'] , label = 'MLPR (RMSE mean score:{})'.format( np.round(np.mean(nested_cv['MLP']),2)))
plt.ylabel('RMSE')
plt.xlabel('Outer loop test well')
plt.xticks([0,1,2,3],labels = ['BOJE-2C', 'JUDE-1', 'SIAH_NE-01X', 'VALDEMAR-2H'])
plt.legend(fontsize = 13)

plt.text(x = plt.xlim()[0]-.2,   y = plt.ylim()[1]+0.01, s = 'Figure 9',fontsize=24)
#print('mean score LR :' , np.mean(nested_score_LR ) , 'Std:', np.std(nested_score_LR ))
#print('mean score RFR:' , np.mean(nested_score_RFR) , 'Std:', np.std(nested_score_RFR))
#print('mean score SVR:' , np.mean(nested_score_SVR) , 'Std:', np.std(nested_score_SVR))
#print('mean score MLP:' , np.mean(nested_score_MLP) , 'Std:', np.std(nested_score_MLP))
##plt.savefig(fig_path+'nested_cv.pdf',format='pdf', bbox_inches='tight')

# Predictions on BO-2X and BO-3X

In [ ]:
estimator_lr = Pipeline(steps=[('std', StandardScaler()), ('LR', LinearRegression())])

estimator_rfr = Pipeline(steps=[('RFR',
                 RandomForestRegressor(max_depth=20, min_samples_leaf=7,
                                       n_estimators=800, n_jobs=5,
                                       random_state=42))])

estimator_svr = Pipeline(steps=[('std', StandardScaler()), ('SVR', SVR(C=0.01, gamma=0.1))])

estimator_mlp = Pipeline(steps=[('std', StandardScaler()),
                ('MLP',
                 MLPRegressor(alpha=0.01, early_stopping=True,
                              hidden_layer_sizes=(8, 8), max_iter=4000,
                              random_state=42, solver='lbfgs',
                              validation_fraction=0.2))])

In [ ]:
# LINEAR REGRESSION
#logo = LeaveOneGroupOut().split(X_train, y_train, groups=wells_train)
#estimator_lr = GridSearchCV(pipe_LR, param_grid_LR, scoring='neg_root_mean_squared_error', cv=logo, verbose = 1, n_jobs=-1)  
estimator_lr.fit(X_train, np.squeeze(y_train))
test_logs['lr'] = estimator_lr.predict(X_test)
print('LR DONE')


# SUPPORTVECTOR REGRESSION
#logo = LeaveOneGroupOut().split(X_train, y_train, groups=wells_train)
#estimator_svr = GridSearchCV(pipe_SVR, param_grid_SVR, scoring='neg_root_mean_squared_error', cv=logo, verbose = 1, n_jobs=-1)  
estimator_svr.fit(X_train, np.squeeze(y_train))
#print(estimator_svr.best_params_)
#pipe_SVR.fit(X_train, y_train)

test_logs['svr'] = estimator_svr.predict(X_test)
print('SVR DONE')

# RANDOMFOREST
#logo = LeaveOneGroupOut().split(X_train, y_train, groups=wells_train)
#pipe_RFR = Pipeline([('s',rfe), ('RFR', regr2)])
#estimator_rfr = GridSearchCV(pipe_RFR, param_grid_RFR, scoring='neg_root_mean_squared_error', cv=logo, verbose = 1, n_jobs=-1)
estimator_rfr.fit(X_train, np.squeeze(y_train))
#pipe_RFR.set_params(**estimator_rfr.best_params_)
#pipe_RFR.fit(X_train, y_train)
test_logs['rfr'] = estimator_rfr.predict(X_test)
print('RFR DONE')

#logo = LeaveOneGroupOut().split(X_train, y_train, groups=wells_train)
#pipe_RFR = Pipeline([('s',rfe), ('RFR', regr2)])
#estimator_mlp = GridSearchCV(pipe_MLP, param_grid_MLP, scoring='neg_root_mean_squared_error', cv=logo, verbose = 1, n_jobs=-1)
estimator_mlp.fit(X_train, np.squeeze(y_train))
#pipe_MLP.set_params(**estimator_mlp.best_params_)
#pipe_MLP.fit(X_train, y_train)
test_logs['mlp'] = estimator_mlp.predict(X_test)
print('MLP DONE')


test_logs.to_csv(path_project+'wells_pred.csv',index=False)

In [ ]:
scores = np.zeros([4,4])

scores[0,0] = mean_squared_error(y_train , estimator_lr.predict(X_train), squared=False)
scores[1,0] = mean_squared_error(y_train ,estimator_svr.predict(X_train), squared=False)
scores

In [ ]:
scores = np.zeros([5,5])

scores[0,0] =np.round( mean_squared_error(y_train , estimator_lr.predict(X_train), squared=False),2 ) 
scores[1,0] =np.round(mean_squared_error(y_train ,estimator_rfr.predict(X_train), squared=False),2 ) 
scores[2,0] =np.round( mean_squared_error(y_train ,estimator_svr.predict(X_train), squared=False),2 ) 
scores[3,0] =np.round( mean_squared_error(y_train ,estimator_mlp.predict(X_train), squared=False),2 ) 


#scores[0,1] = mean_squared_error(test_logs['true'],test_logs['lr'],  squared=False)
#scores[1,1] = mean_squared_error(test_logs['true'],test_logs['svr'], squared=False)
#scores[2,1] = mean_squared_error(test_logs['true'],test_logs['rfr'], squared=False)
#scores[3,1] = mean_squared_error(test_logs['true'],test_logs['mlp'], squared=False)


bo2x_len = df.loc[df['UWI'] == 'BO-2X'].shape[0]
bo3x_len = df.loc[df['UWI'] == 'BO-3X'].shape[0]

scores[0,2] = np.round(mean_squared_error(test_logs['true'][:bo2x_len],test_logs['lr'][:bo2x_len],  squared=False),2)
scores[1,2] = np.round(mean_squared_error(test_logs['true'][:bo2x_len],test_logs['rfr'][:bo2x_len], squared=False),2)
scores[2,2] = np.round(mean_squared_error(test_logs['true'][:bo2x_len],test_logs['svr'][:bo2x_len], squared=False),2)
scores[3,2] = np.round(mean_squared_error(test_logs['true'][:bo2x_len],test_logs['mlp'][:bo2x_len], squared=False),2)

scores[0,3] = np.round(mean_squared_error(test_logs['true'][bo2x_len:],test_logs['lr'][bo2x_len:],  squared=False),2)
scores[1,3] = np.round(mean_squared_error(test_logs['true'][bo2x_len:],test_logs['rfr'][bo2x_len:], squared=False),2)
scores[2,3] = np.round(mean_squared_error(test_logs['true'][bo2x_len:],test_logs['svr'][bo2x_len:], squared=False),2)
scores[3,3] = np.round(mean_squared_error(test_logs['true'][bo2x_len:],test_logs['mlp'][bo2x_len:], squared=False),2)

print('LR:',mean_squared_error( test_logs['true'],test_logs['lr'], squared=False))
print('SVR:',mean_squared_error(test_logs['true'],test_logs['rfr'], squared=False))
print('RFR:',mean_squared_error(test_logs['true'],test_logs['svr'], squared=False))
print('MLP:',mean_squared_error(test_logs['true'],test_logs['mlp'], squared=False))


scores[0,1] = np.round((scores[0,2] + scores[0,3]) / 2 , 2)
scores[1,1] = np.round((scores[1,2] + scores[1,3]) / 2 , 2)
scores[2,1] = np.round((scores[2,2] + scores[2,3]) / 2 , 2)
scores[3,1] = np.round((scores[3,2] + scores[3,3]) / 2 , 2)

scores[0,4] = np.round(mean_squared_error(test_logs['true'][tux_start:bo2x_len],test_logs['lr'][tux_start:bo2x_len],  squared=False),2)
scores[1,4] = np.round(mean_squared_error(test_logs['true'][tux_start:bo2x_len],test_logs['rfr'][tux_start:bo2x_len], squared=False),2)
scores[2,4] = np.round(mean_squared_error(test_logs['true'][tux_start:bo2x_len],test_logs['svr'][tux_start:bo2x_len], squared=False),2)
scores[3,4] = np.round(mean_squared_error(test_logs['true'][tux_start:bo2x_len],test_logs['mlp'][tux_start:bo2x_len], squared=False),2)
scores[4,4] = 0.04 #mean_squared_error(bo2x['DTS'][tux_idx:],bo2x['Vs_IF'][tux_idx:], squared=False)  

scores = pd.DataFrame(data = scores, columns = ['Mean RMSE train','Mean RMSE test','BO-2X', 'BO-3X','BO-2X Tuxen'], index=['LR','RFR','SVR','MLPR','RPM'])
scores.to_excel(fig_path + 'scores.xlsx')#nested_cv.to_csv(path_project+'nested_cv.csv',index=False)

In [ ]:
scores

In [ ]:
(0.09 + 0.05) /2

In [ ]:
tux_start = df.loc[(df['UWI'] == 'BO-2X') & (df['Formation'] == 'U.Tuxen-1') ].index[0]
tux_end = bo2x_len
#loc[(bo2x['Formation'] == 'U.Tuxen-1')].index[0]

In [ ]:
df.loc[(df['UWI'] == 'BO-2X') & (df['Formation'] == 'L.Tuxen-1') ]

In [ ]:
df.loc[df['UWI'] == 'BO-3X'].shape[0]

In [ ]:
plt.figure(figsize=(8,3))
plt.plot(test_logs['true'], lw=3,color='k')
plt.plot(test_logs['lr'])
plt.plot(test_logs['svr'])
plt.plot(test_logs['rfr'])
plt.plot(test_logs['mlp'])


print('LR:',mean_squared_error(test_logs['true'],test_logs['lr'], squared=False))
print('SVR:',mean_squared_error(test_logs['true'],test_logs['svr'], squared=False))
print('RFR:',mean_squared_error(test_logs['true'],test_logs['rfr'], squared=False))
print('MLP:',mean_squared_error(test_logs['true'],test_logs['mlp'], squared=False))

In [ ]:
estimator_rfr.best_params_


In [ ]:
estimator_svr.best_params_

In [ ]:
estimator_mlp.best_params_

# Predictions on BO-2X and BO-3X

# Tensorflow

In [ ]:
#from tensorflow.keras import sequential

#class CustomCallback(keras.callbacks.Callback):
#
#    def on_epoch_end(self, epoch, logs=None):
#        y_test_pred = self.model.predict(X_test)
#        mse = mean_squared_error(y_test, y_test_pred)
#        print(f"Epoch {epoch}: {mse:.2f}")
        
        
def create_model(kernel_initializer='normal', activation='relu'):
    model = keras.Sequential()
    model.add(layers.Dense(100,input_dim=len(features),activation=activation,kernel_initializer=kernel_initializer
                           , kernel_regularizer=regularizers.l2(0.00001)))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(100,activation=activation, kernel_regularizer=regularizers.l2(0.00001)))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(50,activation='relu', kernel_regularizer=regularizers.l2(0.00001)))
    model.add(layers.Dense(1 ,activation=activation))

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                 optimizer=tf.keras.optimizers.Adam(0.001))

    return model

# wrap the model using the function you created
clf = KerasRegressor(model=create_model,verbose=0, epochs=200, batch_size=1, fit__validation_data = (X_test,y_test)
                    , validation_batch_size = 1)

# just create the pipeline
pipeline_MLP = Pipeline([('std', StandardScaler()),
    ('clf',clf)
])

pipeline_MLP.fit(X_train, y_train)
plt.figure()
plt.plot(pipeline_MLP.steps[1][1].history_['loss'], label = 'test')
plt.plot(pipeline_MLP.steps[1][1].history_['val_loss'], label = 'val')
plt.legend()
test_logs['mlp'] = pipeline_MLP.predict(X_test)

plt.figure()
plt.plot(test_logs['true'] , label='true')
plt.plot(test_logs['mlp'], label='pred')
plt.legend()
print('MLP:',mean_squared_error(test_logs['true'],test_logs['mlp'], squared=False))

#test_logs['mlp'] = pipeline_MLP.predict(X_test)
#plt.plot(test_logs['true'])
#
#plt.plot(test_logs['mlp'])
#
#print('MLP:',mean_squared_error(test_logs['true'],test_logs['mlp'], squared=False))

In [ ]:
class CustomCallback(keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        y_val_pred = self.model.predict(x_val)
        mse = mean_squared_error(y_val, y_val_pred)
        print(f"Epoch {epoch}: {mse:.2f}")
        
        
#from tensorflow.keras import sequential
def create_model(kernel_initializer='normal', activation='relu'):
    model = keras.Sequential()
    model.add(layers.Dense(5,input_dim=len(features),activation=activation,kernel_initializer=kernel_initializer, kernel_regularizer=regularizers.l2(0.005)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10,activation=activation,kernel_initializer=kernel_initializer, kernel_regularizer=regularizers.l2(0.005)))
#    model.add(layers.Dense(3,activation='relu',kernel_initializer=kernel_initializer, kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dense(1 ,activation=activation))

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                 optimizer=tf.keras.optimizers.Adam(0.1))

    return model

# wrap the model using the function you created
clf = KerasRegressor(model=create_model,verbose=1)

# just create the pipeline
param_grid_MLP = [{ 'clf__batch_size': [10, 20],
 'clf__epochs': [10, 20]}]

logo = LeaveOneGroupOut().split(X_train, y_train, groups=wells_train)


grid = GridSearchCV(estimator=pipeline_MLP, param_grid=param_grid_MLP, n_jobs=-1, cv=logo, return_train_score=True, scoring='neg_root_mean_squared_error')
grid_result = grid.fit(X_train, np.squeeze(y_train))

#clf_mlp.fit(X_train, np.squeeze(y_train))
#test_logs['mlp'] = clf_mlp.predict(X_test)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

test_scores = grid_result.cv_results_['mean_test_score']
train_scores = grid_result.cv_results_['mean_train_score'] 
#'mean_test_score': array([-0.15079611, -0.15745158, -0.12269899, -0.12267689]),
# 'std_test_score': array([0.05027158, 0.03339824, 0.0240863 , 0.01774399]),
plt.plot(test_scores*-1, label='validation')
plt.plot(train_scores*-1, label='train')
plt.legend(loc='best')
plt.show()

In [ ]:
pipeline_MLP.set_params(**grid_result.best_params_)
pipeline_MLP.fit(X_train, y_train)


In [ ]:
history = pipeline_MLP.fit(
    X_train,
    y_train)

In [ ]:
test = pipeline_MLP.predict(X_test)

In [ ]:
plt.plot(history.steps[1][1].history_['loss'])

# CrossVal for Keras

In [ ]:
class KerasRegressorMod(KerasRegressor):
    def fit(self, X, y, sample_weight=None, validation_data=None):
        self.fit__validation_data = validation_data
        super().fit(X, y, sample_weight)
        vars(self).pop('fit__validation_data')
        return self
    
clf = KerasRegressorMod(model=create_model,verbose=1)

# just create the pipeline
pipeline_MLP = Pipeline([('std', StandardScaler()),
    ('clf',clf)])

In [ ]:
history = pipeline_MLP.fit(X_train, y_train, clf__validation_data=(X_test, y_test))

In [ ]:
history.steps[1][1].history_

In [ ]:
plt.plot(history.steps[1][1].history_['loss'])

In [ ]:
X_tr = df.loc[df['UWI'].isin(train_wells), features]
y_tr = df.loc[df['UWI'].isin(train_wells), target]
X_ts = df.loc[df['UWI'].isin(test_wells), features]
y_ts = df.loc[df['UWI'].isin(test_wells), target]

logo = LeaveOneGroupOut()
k=0

nested_score_LR = []
nested_score_RFR = []
nested_score_SVR = []
nested_score_MLP = []

k=0
for train_index, test_index in logo.split(X_tr, y_tr, wells_train):
    print('Iter #:', k)
    X_trn, X_val = X_tr.iloc[train_index], X_tr.iloc[test_index]
    y_trn, y_val = y_tr.iloc[train_index], y_tr.iloc[test_index]
    
    scale = StandardScaler()
    X_trn = scale.fit_transform(X_trn)

    #This is the change
    X_val = scale.transform(X_val)
    
    clf.fit(X_trn, y_trn)
    test = clf.predict(X_val)
    #history = pipeline_MLP.fit(X_trn, y_trn, clf__validation_data=(X_val, y_val))
    
    plt.figure()
    plt.plot(test)
    plt.plot(np.array(y_val))
    
    #print('test well :', df.loc[X_val.index[-1]]['UWI'])
    k+=1

In [ ]:
clf.history_

In [ ]:
history.steps[1][1].history_

In [ ]:
#4.77**2

In [ ]:
from uncertainties import ufloat
from pint import UnitRegistry 
4.77
#RMSE of test data (#1 DTC): 4.77
#RMSE of test data (#2 DTS): 17.28

DT = ufloat(200,(17.28)) *u.microsecond / u.ft

format((1 / DT).to('km/s'), '.2f')

In [ ]:
0.3048 * (1e6/(200-17.28/2) - 1e6/(200+17.28/2))

In [ ]:
0.3048 * (1e6/150 - 1e6/159.676)

# Nested CV (LeaveOneGroupOut)

In [ ]:
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR


regr1 = LinearRegression()
regr2 = RandomForestRegressor(random_state=42)
regr3 = SVR()
regr4 = MLPRegressor(random_state=42)


# Building the pipelines
pipe_LR = Pipeline([('std', StandardScaler()),
                  ('LR', regr1)])

pipe_RFR = Pipeline([('RFR', regr2)])

pipe_SVR = Pipeline([('std', StandardScaler()),
                  ('SVR', regr3)])


pipe_MLP = Pipeline([('std', StandardScaler()), ('MLP', regr4)])


# Setting up the parameter grids
param_grid_LR = [{}]

param_grid_RFR = [{'RFR__max_depth': list(range(1, 10)) + [None],
                  'RFR__n_estimators': [100,200,300,400] }]

param_grid_SVR = [{'SVR__C': np.power(10., np.arange(-4, 4)),
                  'SVR__gamma': [1, 0.1, 0.01, 0.001, 0.0001]}]

param_grid_MLP = [{"MLP__hidden_layer_sizes": [(100), (10,10), (20,20), (50,50), (100,100)],
                  "MLP__activation": ["relu"],
                  'MLP__learning_rate': ['adaptive'],
                  'MLP__solver' : ['adam'],
                  "MLP__alpha": np.power(10., np.arange(-6, 0)),
                  "MLP__max_iter": [4000]}] #np.arange(100,8000,500)

In [ ]:
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

X_tr = df.loc[df['UWI'].isin(train_wells), features]
y_tr = df.loc[df['UWI'].isin(train_wells), target]
X_ts = df.loc[df['UWI'].isin(test_wells), features]
y_ts = df.loc[df['UWI'].isin(test_wells), target]

logo = LeaveOneGroupOut()
k=0

nested_score_LR = []
nested_score_RFR = []
nested_score_SVR = []
nested_score_MLP = []

for train_index, test_index in logo.split(X_tr, y_tr, wells_train):
    print('Iter #:', k)
    X_trn, X_tst = X_tr.iloc[train_index], X_tr.iloc[test_index]
    y_trn, y_tst = y_tr.iloc[train_index], y_tr.iloc[test_index]
    
    print('test well :', df.loc[X_tst.index[-1]]['UWI'])


    wells_train_inner = train_wells.copy()
    wells_train_inner.remove(df.loc[X_tst.index[-1]]['UWI'])
    
    print('training wells:', wells_train_inner)
    
    X_tr_ = df.loc[df['UWI'].isin(wells_train_inner), features].values
    y_tr_ = np.squeeze(df.loc[df['UWI'].isin(wells_train_inner), target].values)


    wells_train_inner = df.loc[df['UWI'].isin(wells_train_inner), 'UWI'].values
    
    #LR
    gcv_LR = GridSearchCV(pipe_LR, param_grid_LR, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut())
    results_LR = gcv_LR.fit(X_tr_, y_tr_, groups=wells_train_inner)
    model_LR = results_LR.best_estimator_.fit(X_tr_, y_tr_)
    model_score_LR = model_LR.predict(X_tst)
    nested_score_LR.append(mean_squared_error(y_tst,model_score_LR, squared=False))
    
    #RFR
    gcv_RFR = GridSearchCV(pipe_RFR, param_grid_RFR, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut())
    results_RFR = gcv_RFR.fit(X_tr_, y_tr_, groups=wells_train_inner)
    model_RFR = results_RFR.best_estimator_.fit(X_tr_, y_tr_)
    model_score_RFR = model_RFR.predict(X_tst)
    nested_score_RFR.append(mean_squared_error(y_tst,model_score_RFR, squared=False))
    
    
    #SVR
    gcv_SVR = GridSearchCV(pipe_SVR, param_grid_SVR, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut())
    results_SVR = gcv_SVR.fit(X_tr_, y_tr_, groups=wells_train_inner)
    model_SVR = results_SVR.best_estimator_.fit(X_tr_, y_tr_)
    model_score_SVR = model_SVR.predict(X_tst)
    nested_score_SVR.append(mean_squared_error(y_tst,model_score_SVR, squared=False))
    
    #MLP
    gcv_MLP = GridSearchCV(pipe_MLP, param_grid_MLP, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut())
    results_MLP = gcv_MLP.fit(X_tr_, y_tr_, groups=wells_train_inner)
    model_MLP = results_MLP.best_estimator_.fit(X_tr_, y_tr_)
    model_score_MLP = model_MLP.predict(X_tst)
    nested_score_MLP.append(mean_squared_error(y_tst,model_score_MLP, squared=False))
    
    k+=1



In [ ]:
plt.plot(np.arange(4), nested_score_LR  , label = 'LR')
plt.plot(np.arange(4), nested_score_RFR , label = 'RFR')
plt.plot(np.arange(4), nested_score_SVR , label = 'SVR')
plt.plot(np.arange(4), nested_score_MLP , label = 'MLP')
plt.ylabel('RMSE')
plt.xlabel('Validation Well')
plt.xticks([0,1,2,3],labels = ['BOJE-2C', 'JUDE-1', 'SIAH_NE-01X', 'VALDEMAR-2H'])
plt.legend()


plt.fill_between(
    np.arange(4),
    nested_score_LR - np.std(nested_score_LR ),
    nested_score_LR + np.std(nested_score_LR ),
    alpha=0.1,
    color="C0"
)

plt.fill_between(
    np.arange(4),
    nested_score_RFR - np.std(nested_score_RFR ),
    nested_score_RFR + np.std(nested_score_RFR ),
    alpha=0.1,
    color="C1"
)

plt.fill_between(
    np.arange(4),
    nested_score_SVR - np.std(nested_score_SVR ),
    nested_score_SVR + np.std(nested_score_SVR ),
    alpha=0.1,
    color="C2"
)

plt.fill_between(
    np.arange(4),
    nested_score_MLP - np.std(nested_score_MLP ),
    nested_score_MLP + np.std(nested_score_MLP ),
    alpha=0.1,
    color="C2"
)

print('mean score LR :' , np.mean(nested_score_LR ) , 'Std:', np.std(nested_score_LR ))
print('mean score RFR:' , np.mean(nested_score_RFR) , 'Std:', np.std(nested_score_RFR))
print('mean score SVR:' , np.mean(nested_score_SVR) , 'Std:', np.std(nested_score_SVR))
print('mean score MLP:' , np.mean(nested_score_MLP) , 'Std:', np.std(nested_score_MLP))


In [ ]:
nested_score_MLP

In [ ]:
print('mean score LR :' , np.mean(nested_score_LR ) , 'Std:', np.std(nested_score_LR ))
print('mean score RFR:' , np.mean(nested_score_RFR) , 'Std:', np.std(nested_score_RFR))
print('mean score SVR:' , np.mean(nested_score_SVR) , 'Std:', np.std(nested_score_SVR))
print('mean score MLP:' , np.mean(nested_score_MLP) , 'Std:', np.std(nested_score_MLP))

In [ ]:
results_MLP.best_estimator_

In [ ]:
#logo = LeaveOneGroupOut().split(X_train, y_train, groups=wells_train)
#nested_score = cross_val_score(results.best_estimator_, X_train, y_train, scoring='neg_root_mean_squared_error', cv=logo, groups=wells_train)

# Nested CV with RFE

In [ ]:
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import feature_selection
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor


rfe = RFE(estimator=DecisionTreeRegressor(), n_features_to_select=10)


regr1 = LinearRegression()
regr2 = RandomForestRegressor(random_state=42)
regr3 = SVR()
regr4 = MLPRegressor(random_state=42)


# Building the pipelines
pipe_LR = Pipeline([('s',rfe) , ('std', StandardScaler()),
                  ('LR', regr1)])

pipe_RFR = Pipeline([('s',rfe), ('RFR', regr2)])

pipe_SVR = Pipeline([('s',rfe) , ('std', StandardScaler()),
                  ('SVR', regr3)])


pipe_MLP = Pipeline([('s',rfe), ('std', StandardScaler()), ('MLP', regr4)])


# Setting up the parameter grids
param_grid_LR = [{}]

param_grid_RFR = [{'RFR__max_depth': list(range(1, 10)) + [None],
                  'RFR__n_estimators': [100,200,300,400] }]

param_grid_SVR = [{'SVR__C': np.power(10., np.arange(-4, 4)),
                  'SVR__gamma': [1, 0.1, 0.01, 0.001, 0.0001]}]

param_grid_MLP = [{"MLP__hidden_layer_sizes": [(100), (10,10), (20,20), (50,50), (100,100)],
                  "MLP__activation": ["relu"],
                  'MLP__learning_rate': ['adaptive'],
                  'MLP__solver' : ['adam'],
                  "MLP__alpha": np.power(10., np.arange(-6, 0)),
                  "MLP__max_iter": [4000]}] #np.arange(100,8000,500)

In [ ]:
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

X_tr = df.loc[df['UWI'].isin(train_wells), features]
y_tr = df.loc[df['UWI'].isin(train_wells), target]
X_ts = df.loc[df['UWI'].isin(test_wells), features]
y_ts = df.loc[df['UWI'].isin(test_wells), target]

logo = LeaveOneGroupOut()
k=0

nested_score_LR = []
nested_score_RFR = []
nested_score_SVR = []
nested_score_MLP = []

for train_index, test_index in logo.split(X_tr, y_tr, wells_train):
    print('Iter #:', k)
    X_trn, X_tst = X_tr.iloc[train_index], X_tr.iloc[test_index]
    y_trn, y_tst = y_tr.iloc[train_index], y_tr.iloc[test_index]
    
    print('test well :', df.loc[X_tst.index[-1]]['UWI'])


    wells_train_inner = train_wells.copy()
    wells_train_inner.remove(df.loc[X_tst.index[-1]]['UWI'])
    
    print('training wells:', wells_train_inner)
    
    X_tr_ = df.loc[df['UWI'].isin(wells_train_inner), features].values
    y_tr_ = np.squeeze(df.loc[df['UWI'].isin(wells_train_inner), target].values)


    wells_train_inner = df.loc[df['UWI'].isin(wells_train_inner), 'UWI'].values
    
    #LR
    gcv_LR = GridSearchCV(pipe_LR, param_grid_LR, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut())
    results_LR = gcv_LR.fit(X_tr_, y_tr_, groups=wells_train_inner)
    model_LR = results_LR.best_estimator_.fit(X_tr_, y_tr_)
    model_score_LR = model_LR.predict(X_tst)
    nested_score_LR.append(mean_squared_error(y_tst,model_score_LR, squared=False))
    
    #RFR
    gcv_RFR = GridSearchCV(pipe_RFR, param_grid_RFR, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut())
    results_RFR = gcv_RFR.fit(X_tr_, y_tr_, groups=wells_train_inner)
    model_RFR = results_RFR.best_estimator_.fit(X_tr_, y_tr_)
    model_score_RFR = model_RFR.predict(X_tst)
    nested_score_RFR.append(mean_squared_error(y_tst,model_score_RFR, squared=False))
    
    
    #SVR
    gcv_SVR = GridSearchCV(pipe_SVR, param_grid_SVR, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut())
    results_SVR = gcv_SVR.fit(X_tr_, y_tr_, groups=wells_train_inner)
    model_SVR = results_SVR.best_estimator_.fit(X_tr_, y_tr_)
    model_score_SVR = model_SVR.predict(X_tst)
    nested_score_SVR.append(mean_squared_error(y_tst,model_score_SVR, squared=False))
    
    #MLP
    gcv_MLP = GridSearchCV(pipe_MLP, param_grid_MLP, scoring='neg_root_mean_squared_error', cv=LeaveOneGroupOut())
    results_MLP = gcv_MLP.fit(X_tr_, y_tr_, groups=wells_train_inner)
    model_MLP = results_MLP.best_estimator_.fit(X_tr_, y_tr_)
    model_score_MLP = model_MLP.predict(X_tst)
    nested_score_MLP.append(mean_squared_error(y_tst,model_score_MLP, squared=False))
    
    k+=1



In [ ]:
plt.plot(np.arange(4), nested_score_LR  , label = 'LR')
plt.plot(np.arange(4), nested_score_RFR , label = 'RFR')
plt.plot(np.arange(4), nested_score_SVR , label = 'SVR')
plt.plot(np.arange(4), nested_score_MLP , label = 'MLP')
plt.ylabel('RMSE')
plt.xlabel('Validation Well')
plt.xticks([0,1,2,3],labels = ['BOJE-2C', 'JUDE-1', 'SIAH_NE-01X', 'VALDEMAR-2H'])
plt.legend()


plt.fill_between(
    np.arange(4),
    nested_score_LR - np.std(nested_score_LR ),
    nested_score_LR + np.std(nested_score_LR ),
    alpha=0.1,
    color="C0"
)

plt.fill_between(
    np.arange(4),
    nested_score_RFR - np.std(nested_score_RFR ),
    nested_score_RFR + np.std(nested_score_RFR ),
    alpha=0.1,
    color="C1"
)

plt.fill_between(
    np.arange(4),
    nested_score_SVR - np.std(nested_score_SVR ),
    nested_score_SVR + np.std(nested_score_SVR ),
    alpha=0.1,
    color="C2"
)

plt.fill_between(
    np.arange(4),
    nested_score_MLP - np.std(nested_score_MLP ),
    nested_score_MLP + np.std(nested_score_MLP ),
    alpha=0.1,
    color="C2"
)

print('mean score LR :' , np.mean(nested_score_LR ) , 'Std:', np.std(nested_score_LR ))
print('mean score RFR:' , np.mean(nested_score_RFR) , 'Std:', np.std(nested_score_RFR))
print('mean score SVR:' , np.mean(nested_score_SVR) , 'Std:', np.std(nested_score_SVR))
print('mean score MLP:' , np.mean(nested_score_MLP) , 'Std:', np.std(nested_score_MLP))


In [ ]:
mean score LR : 0.12636835831088702 Std: 0.03289854152198718
mean score RFR: 0.11639220359273304 Std: 0.026268406898577686
mean score SVR: 0.10196428215112074 Std: 0.03725603668502954
mean score MLP: 0.10148377149739016 Std: 0.05692889168846151
        
        
mean score LR : 0.11630283448532498 Std: 0.059124087596068776
mean score RFR: 0.11248545517280101 Std: 0.023239336498082918
mean score SVR: 0.11043857509214851 Std: 0.047871997594650495
mean score MLP: 0.08996400743992972 Std: 0.045256619671477455